## Read the csv file, drop the duplicate(based on converstaionID) and remove unncessary column

In [2]:
import pandas as pd
import os
from utils import show_df

os.chdir('D:\\Project\\Twitter_depression_detector\\data\\tweet_trend_dataset')
os.getcwd()


'D:\\Project\\Twitter_depression_detector\\data\\tweet_trend_dataset'

In [3]:
# Reads the json generated from the CLI commands above and creates a pandas dataframe
tweets_df = pd.read_csv('depression_tweets_2020-03-01.csv')
tweets_df=tweets_df.drop_duplicates(subset=['conversationId'])
tweets_df=tweets_df.drop(columns=['Unnamed: 0'],axis=1)


## Search for all the hastags in tweet using regex

In [4]:
tweets_df['hashtags']=tweets_df.content.str.findall(r'#.*?(?=\s|$)')
print(tweets_df['hashtags'])

show_df(tweets_df)

0                                                       []
1                                                       []
2                                                       []
3                                                       []
4                                                       []
                               ...                        
13276                                                   []
13277                                                   []
13278    [#GratefulnessSunday, #weonlydopositive, #ment...
13279    [#JinShinJyutsu, #energyhealing, #quantum, #he...
13280                                                   []
Name: hashtags, Length: 11768, dtype: object


Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [], 'inp…

## get Count the hashtags and remove the hastags related to medical terms

In [5]:
tweets_df.hashtags.value_counts().head(20)
medical_terms = ["#mentalhealth", "#health", "#happiness", "#mentalillness", "#happy", "#joy", "#wellbeing"]
mask1 = tweets_df.hashtags.apply(lambda x: any(item for item in medical_terms if item in x))
print(tweets_df[mask1].content.tail())
tweets_df[mask1==False].content.head(10)
tweets_df=tweets_df[mask1==False]

show_df(tweets_df)


13081    A holistic approach to therapy delves into the...
13139    Rants - Episode #3. https://t.co/Jv8iImfBJS vi...
13233    The simplest thing yet it could make a huge di...
13261    New blog online #mentalhealth #anxiety #bpd #d...
13278    You have everything you need already - stay gr...
Name: content, dtype: object


Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [], 'inp…

## remove tweets which contains too many hastags

In [6]:
mask2 = tweets_df.hashtags.apply(lambda x: len(x) < 4)
tweets_df=tweets_df[mask2]
tweets_df.hashtags.value_counts().head(20)

show_df(tweets_df)


Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [], 'inp…

## remove tweets with @ mentions as they are sometimes retweets


In [7]:
import ast
print("Len of dataset: ",len(tweets_df))
# the mentioned user were stored as string so converted them to list
tweets_df['mentionedUsers']=[ast.literal_eval(mentioneduser) if type(mentioneduser)!=float else str(mentioneduser) for mentioneduser in tweets_df['mentionedUsers']]
mask3 = tweets_df.mentionedUsers.apply(lambda x: len(x) < 5)
tweets_df = tweets_df[mask3]
print("Len of dataset: ",len(tweets_df))

Len of dataset:  11002
Len of dataset:  10903


In [8]:
show_df(tweets_df)


Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [], 'inp…

## Remove tweets containing URLS as they might be promotional msgs

In [9]:
import ast
print(len(tweets_df))
tweets_df['outlinks']=[ast.literal_eval(outlink) for outlink in tweets_df['outlinks']]
mask4 = tweets_df.outlinks.apply(lambda x: len(x)==0)
tweets_df = tweets_df[mask4]
print(len(tweets_df))

10903
9200


In [10]:
show_df(tweets_df)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [], 'inp…

## Feature engineering, a column featuring count of the mentioneduser

In [23]:
tweets_df['mentionedUserCount']=[len(mentionedUser) if type(mentionedUser)==list else 0 for mentionedUser in tweets_df['mentionedUsers']]

In [24]:
show_df(tweets_df)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [], 'inp…

In [11]:
tweets_df.to_csv('cleaned_tweets.csv')

## see the difference between two columns content and renderedcontent

In [17]:
count=0
for i in range(len(tweets_df)):
    if tweets_df.iloc[i]['renderedContent']==tweets_df.iloc[i]['content']:
        count+=1
print(count)        

9200


## select the useful columns


In [26]:
tweets_df.columns

Index(['url', 'date', 'content', 'renderedContent', 'id', 'user', 'outlinks',
       'tcooutlinks', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'media',
       'retweetedTweet', 'quotedTweet', 'mentionedUsers', 'hashtags',
       'mentionedUserCount'],
      dtype='object')

In [27]:
tweets_cleaned_df=tweets_df[['renderedContent','replyCount','retweetCount','likeCount','quoteCount','mentionedUserCount']]

In [28]:
show_df(tweets_cleaned_df)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [], 'inp…

## Exporting the cleaned tweets

In [32]:
tweets_cleaned_df.to_csv('cleaned_tweets.csv',index=False)